## Get 2018 world cup results and add to original results file provided by instructors

In [1]:
from requests import get
from requests.exceptions import RequestException
from contextlib import closing
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests
import pandas as pd
import pandasql as ps
from datetime import datetime

## 1 - Get the 2018 world cup results from FIFA.com

### 1.1 Group stage results

In [3]:
# get list of all available FIFA ranking pages

wc_results_url = "https://www.fifa.com/worldcup/matches/#knockoutphase"


try:
    page_response = requests.get(wc_results_url, timeout=5)
    
    if page_response.status_code == 200:
        page_content = BeautifulSoup(page_response.content,'lxml')
        wc_results = page_content.find('div', attrs={'class':"fi-matchlist"})
        #print(wc_results)
        dates = [s['data-utcdate'] for s in wc_results.select('.fi-mu__info__datetime')]
        scores = [s.get_text().strip() for s in wc_results.select('.fi-s__scoreText')]
        home_scores = [s.get_text().strip().split('-')[0] for s in wc_results.select('.fi-s__scoreText')]       
        away_scores = [s.get_text().strip().split('-')[1] for s in wc_results.select('.fi-s__scoreText')]    
        away_teams = [s.get_text().strip().split('\n')[0]  for s in wc_results.find_all('div',attrs={'class':'fi-t fi-i--4 away'})]
        home_teams = [s.get_text().strip().split('\n')[0] for s in wc_results.find_all('div',attrs={'class':'fi-t fi-i--4 home'})]
        venues  = [v.get_text().strip() for v in wc_results.find_all('div',attrs={'class':'fi__info__venue'})]

        wc_results_df = pd.DataFrame({
            "date" : dates,
            "home_team" : home_teams,
            "away_team" : away_teams,
            "home_score" : home_scores,
            "away_score" : away_scores,
            "tournament" : "FIFA World Cup",
            "city" : venues,
            "country" : "Russia"
        })
        
    else:
        print(page_response.status_code)

except requests.Timeout as e:
    print('Timeout occurred for requested page: ' + fifa_url)
    print(str(e))

In [4]:
wc_results_df

,date,home_team,away_team,home_score,away_score,tournament,city,country
0,2018-06-14T03:00:00.000Z,Russia,Saudi Arabia,5,0,FIFA World Cup,Moscow,Russia
1,2018-06-15T12:00:00.000Z,Egypt,Uruguay,0,1,FIFA World Cup,Ekaterinburg,Russia
2,2018-06-15T03:00:00.000Z,Morocco,IR Iran,0,1,FIFA World Cup,St. Petersburg,Russia
3,2018-06-15T06:00:00.000Z,Portugal,Spain,3,3,FIFA World Cup,Sochi,Russia
4,2018-06-16T10:00:00.000Z,France,Australia,2,1,FIFA World Cup,Kazan,Russia
5,2018-06-16T01:00:00.000Z,Argentina,Iceland,1,1,FIFA World Cup,Moscow,Russia
6,2018-06-16T04:00:00.000Z,Peru,Denmark,0,1,FIFA World Cup,Saransk,Russia
7,2018-06-16T07:00:00.000Z,Croatia,Nigeria,2,0,FIFA World Cup,Kaliningrad,Russia
8,2018-06-17T12:00:00.000Z,Costa Rica,Serbia,0,1,FIFA World Cup,Samara,Russia
9,2018-06-17T03:00:00.000Z,Germany,Mexico,0,1,FIFA World Cup,Moscow,Russia


### 1.2 Knockout stage results

In [6]:
# get list of all available FIFA ranking pages

wc_results_url = "https://www.fifa.com/worldcup/matches/#knockoutphase"


try:
    page_response = requests.get(wc_results_url, timeout=5)
    
    if page_response.status_code == 200:
        page_content = BeautifulSoup(page_response.content,'lxml')
        
        wc_results = page_content.find('div', attrs = {'data-tab' : 'knockoutphase'})

        dates = [s['data-utcdate'] for s in wc_results.select('.fi-mu__info__datetime')]
        scores = [s.get_text().strip() for s in wc_results.select('.fi-s__scoreText')]
        home_scores = [s.get_text().strip().split('-')[0] for s in wc_results.select('.fi-s__scoreText')]       
        away_scores = [s.get_text().strip().split('-')[1] for s in wc_results.select('.fi-s__scoreText')]    
        away_teams = [s.get_text().strip().split('\n')[0]  for s in wc_results.find_all('div',attrs={'class':'fi-t fi-i--4 away'})]
        home_teams = [s.get_text().strip().split('\n')[0] for s in wc_results.find_all('div',attrs={'class':'fi-t fi-i--4 home'})]
        venues  = [v.get_text().strip() for v in wc_results.find_all('div',attrs={'class':'fi__info__venue'})]

        wc_results2_df = pd.DataFrame({
            "date" : dates,
            "home_team" : home_teams,
            "away_team" : away_teams,
            "home_score" : home_scores,
            "away_score" : away_scores,
            "tournament" : "FIFA World Cup",
            "city" : venues,
            "country" : "Russia"
        })
        
    else:
        print(page_response.status_code)

except requests.Timeout as e:
    print('Timeout occurred for requested page: ' + fifa_url)
    print(str(e))

In [7]:
wc_results2_df

,date,home_team,away_team,home_score,away_score,tournament,city,country
0,2018-06-30T02:00:00.000Z,France,Argentina,4,3,FIFA World Cup,Kazan,Russia
1,2018-06-30T06:00:00.000Z,Uruguay,Portugal,2,1,FIFA World Cup,Sochi,Russia
2,2018-07-01T02:00:00.000Z,Spain,Russia,1,1,FIFA World Cup,Moscow,Russia
3,2018-07-01T06:00:00.000Z,Croatia,Denmark,1,1,FIFA World Cup,Nizhny Novgorod,Russia
4,2018-07-02T02:00:00.000Z,Brazil,Mexico,2,0,FIFA World Cup,Samara,Russia
5,2018-07-02T06:00:00.000Z,Belgium,Japan,3,2,FIFA World Cup,Rostov-On-Don,Russia
6,2018-07-03T02:00:00.000Z,Sweden,Switzerland,1,0,FIFA World Cup,St. Petersburg,Russia
7,2018-07-03T06:00:00.000Z,Colombia,England,1,1,FIFA World Cup,Moscow,Russia
8,2018-07-06T02:00:00.000Z,Uruguay,France,0,2,FIFA World Cup,Nizhny Novgorod,Russia
9,2018-07-06T06:00:00.000Z,Brazil,Belgium,1,2,FIFA World Cup,Kazan,Russia


### 1.3 - Concatenate group and knockout stage

In [8]:
wc_results_df = pd.concat([wc_results_df , wc_results2_df])

In [9]:
wc_results_df

,date,home_team,away_team,home_score,away_score,tournament,city,country
0,2018-06-14T03:00:00.000Z,Russia,Saudi Arabia,5,0,FIFA World Cup,Moscow,Russia
1,2018-06-15T12:00:00.000Z,Egypt,Uruguay,0,1,FIFA World Cup,Ekaterinburg,Russia
2,2018-06-15T03:00:00.000Z,Morocco,IR Iran,0,1,FIFA World Cup,St. Petersburg,Russia
3,2018-06-15T06:00:00.000Z,Portugal,Spain,3,3,FIFA World Cup,Sochi,Russia
4,2018-06-16T10:00:00.000Z,France,Australia,2,1,FIFA World Cup,Kazan,Russia
5,2018-06-16T01:00:00.000Z,Argentina,Iceland,1,1,FIFA World Cup,Moscow,Russia
6,2018-06-16T04:00:00.000Z,Peru,Denmark,0,1,FIFA World Cup,Saransk,Russia
7,2018-06-16T07:00:00.000Z,Croatia,Nigeria,2,0,FIFA World Cup,Kaliningrad,Russia
8,2018-06-17T12:00:00.000Z,Costa Rica,Serbia,0,1,FIFA World Cup,Samara,Russia
9,2018-06-17T03:00:00.000Z,Germany,Mexico,0,1,FIFA World Cup,Moscow,Russia


### 1.4 - Add missing columns and clean up

In [10]:
wc_results_df['neutral'] = (wc_results_df['home_team'] != wc_results_df['country'])

In [11]:
wc_results_df['date'] = wc_results_df.date.astype('datetime64').dt.normalize()

In [12]:
wc_results_df.reset_index(drop=True, inplace=True)

In [13]:
wc_results_df

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
0,2018-06-14,Russia,Saudi Arabia,5,0,FIFA World Cup,Moscow,Russia,False
1,2018-06-15,Egypt,Uruguay,0,1,FIFA World Cup,Ekaterinburg,Russia,True
2,2018-06-15,Morocco,IR Iran,0,1,FIFA World Cup,St. Petersburg,Russia,True
3,2018-06-15,Portugal,Spain,3,3,FIFA World Cup,Sochi,Russia,True
4,2018-06-16,France,Australia,2,1,FIFA World Cup,Kazan,Russia,True
5,2018-06-16,Argentina,Iceland,1,1,FIFA World Cup,Moscow,Russia,True
6,2018-06-16,Peru,Denmark,0,1,FIFA World Cup,Saransk,Russia,True
7,2018-06-16,Croatia,Nigeria,2,0,FIFA World Cup,Kaliningrad,Russia,True
8,2018-06-17,Costa Rica,Serbia,0,1,FIFA World Cup,Samara,Russia,True
9,2018-06-17,Germany,Mexico,0,1,FIFA World Cup,Moscow,Russia,True


In [ ]:
wc_results_df.info()

In [14]:
# back up raw data (in case FIFA web site changes)
wc_results_df.to_csv('wc_results_2018.csv') 

## 2 - Combine with original international results file

In [21]:
intl_results = pd.read_csv('data/fifa/international_results.csv')
intl_results.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39638 entries, 0 to 39637
Data columns (total 10 columns):
date          39638 non-null object
home_team     39638 non-null object
away_team     39638 non-null object
home_score    39638 non-null int64
away_score    39638 non-null int64
tournament    39638 non-null object
city          39638 non-null object
country       39638 non-null object
neutral       39638 non-null bool
Unnamed: 9    2516 non-null object
dtypes: bool(1), int64(2), object(7)
memory usage: 2.8+ MB


In [22]:
intl_results['date'] = intl_results.date.astype('datetime64')

In [23]:
intl_results.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39638 entries, 0 to 39637
Data columns (total 10 columns):
date          39638 non-null datetime64[ns]
home_team     39638 non-null object
away_team     39638 non-null object
home_score    39638 non-null int64
away_score    39638 non-null int64
tournament    39638 non-null object
city          39638 non-null object
country       39638 non-null object
neutral       39638 non-null bool
Unnamed: 9    2516 non-null object
dtypes: bool(1), datetime64[ns](1), int64(2), object(6)
memory usage: 2.8+ MB


In [24]:

intl_results = intl_results[intl_results['date'] < datetime.strptime('2018-06-14', '%Y-%m-%d')]
intl_results = intl_results.iloc[:,0:9]
intl_results.head()

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
0,1872-11-30,Scotland,England,0,0,Friendly,Glasgow,Scotland,False
1,1873-03-08,England,Scotland,4,2,Friendly,London,England,False
2,1874-03-07,Scotland,England,2,1,Friendly,Glasgow,Scotland,False
3,1875-03-06,England,Scotland,2,2,Friendly,London,England,False
4,1876-03-04,Scotland,England,3,0,Friendly,Glasgow,Scotland,False


In [30]:
intl_results.tail()

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
39601,2018-06-10,Austria,Brazil,0,3,Friendly,Vienna,Austria,False
39602,2018-06-11,Korea Republic,Senegal,0,2,Friendly,Grödig,Austria,True
39603,2018-06-11,Belgium,Costa Rica,4,1,Friendly,Brussels,Belgium,False
39604,2018-06-12,Japan,Paraguay,4,2,Friendly,Innsbruck,Austria,True
39605,2018-06-12,Poland,Lithuania,4,0,Friendly,Warsaw,Poland,False


In [31]:
full_results = pd.concat([intl_results , wc_results_df])

In [39]:
full_results.head()

,home_team,away_team,home_score,away_score,tournament,city,country,neutral
date,,,,,,,,
1872-11-30,Scotland,England,0,0,Friendly,Glasgow,Scotland,False
1873-03-08,England,Scotland,4,2,Friendly,London,England,False
1874-03-07,Scotland,England,2,1,Friendly,Glasgow,Scotland,False
1875-03-06,England,Scotland,2,2,Friendly,London,England,False
1876-03-04,Scotland,England,3,0,Friendly,Glasgow,Scotland,False


In [33]:
full_results.tail()

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
59,2018-07-07,Russia,Croatia,2,2,FIFA World Cup,Sochi,Russia,False
60,2018-07-10,France,Belgium,1,0,FIFA World Cup,St. Petersburg,Russia,True
61,2018-07-11,Croatia,England,2,1,FIFA World Cup,Moscow,Russia,True
62,2018-07-14,Belgium,England,2,0,FIFA World Cup,St. Petersburg,Russia,True
63,2018-07-15,France,Croatia,4,2,FIFA World Cup,Moscow,Russia,True


In [ ]:
full_results.set_index('date',inplace=True,drop=True)

In [40]:
full_results.head()

,home_team,away_team,home_score,away_score,tournament,city,country,neutral
date,,,,,,,,
1872-11-30,Scotland,England,0,0,Friendly,Glasgow,Scotland,False
1873-03-08,England,Scotland,4,2,Friendly,London,England,False
1874-03-07,Scotland,England,2,1,Friendly,Glasgow,Scotland,False
1875-03-06,England,Scotland,2,2,Friendly,London,England,False
1876-03-04,Scotland,England,3,0,Friendly,Glasgow,Scotland,False


In [41]:
full_results.to_csv('new_intl_results.csv')